# Application d'analyse des données de Formule 1 saison 2023
Ceci est la version 2023

Cette fois, je part de la base de ce que j'a fait la saison précédente et essayer d'avoir
une meilleur data visualisation

Je vais vraiment essayer de séparer entre les batailles principales

## Imports

In [ ]:
import pandas as pd
import numpy as np
import cufflinks as cf
import plotly.express as px
import chart_studio.plotly as py
import seaborn as sns
%matplotlib inline
import matplotlib.pyplot as plt

from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly.graph_objs as go
import json

## Partie Générale

Dans cette partie, je m'intéresse à la vision générale de la saison.
C'est cette partie qui reste très proche de la version 2022

### Backend

In [ ]:
# importation des données
data_all = pd.read_csv("data.csv", sep=",")
data_all["index"] = data_all.groupby("pilot").cumcount()
data_all["index"] += 1

# dataset des points cumulés par pilote
data_points = data_all[["pilot", "points"]].copy()
data_points["cum_points"] = data_points.groupby("pilot").cumsum()
data_points["index"] = data_points.groupby("pilot").cumcount()

# extraction des pilotes par écurie
with open("teams.json") as jsonfile:
    teams_pilot = json.load(jsonfile)
data_team_points = {}
for team in teams_pilot.keys():
    data_team_points[team] = []
    
# pour chaque équipe on fait la somme des points de chaque pilote de l'équipe
for team in teams_pilot.keys():
    s = 0
    j = 0
    for i in range(data_points.shape[0]):
        if data_points.iloc[i]["pilot"] in teams_pilot[team]:
            s += data_points.iloc[i]["points"]
            j += 1
        if j == len(teams_pilot[team]):
            j = 0
            data_team_points[team].append(s)
data_team_points = pd.DataFrame(data_team_points)
test = []
for i in range(data_team_points.shape[0]):
    for team in teams_pilot.keys():
        test.append([team, data_team_points.iloc[i][team]])
data_team_points = pd.DataFrame(test, columns=["team", "points"])
data_team_points["index"] = data_team_points.groupby("team").cumcount()

### Frontend

## Partie Bataille entre pilotes

Ici je m'interesse surtout au bataille entre les pilotes

### Backend

### Frontend

## Partie Bataille entre les écuries

Ici je m'interesse surtout au bataille entre les écuries

### Backend

### Frontend